In [194]:
import pandas as pd
import numpy as np
import catboost
import pickle
from sklearn.metrics import r2_score

In [195]:
def mertic_for_task(y_test, pred):
    score_views = r2_score(y_test["views"], pred[:,0])
    print(f"score_views={score_views}")
    score_depth = r2_score(y_test["depth"],pred[:,1])
    print(f"score_depth={score_depth}")
    score_frp = r2_score(y_test["full_reads_percent"], pred[:,2])
    print(f"score_frp={score_frp}")
    print(f"final_score={0.4 * score_views + 0.3 * score_depth + 0.3 * score_frp}")
    return 0.4 * score_views + 0.3 * score_depth + 0.3 * score_frp

In [196]:
df_train = pd.read_parquet('dataset/train_dataset_train_full_proc.parquet')

In [197]:
features_all = np.zeros((len(df_train), 12 + 
                         len(df_train['embedding_category'].iloc[0]) + 
                         len(df_train['embedding_tags'].iloc[0]) + 
                         len(df_train['embedding_authors'].iloc[0]) + 
                         len(df_train['embedding_title'].iloc[0]) + 
                         len(df_train['embedding_document_id_1'].iloc[0]) +
                         len(df_train['embedding_document_id_2'].iloc[0]) +
                         len(df_train['embedding_document_id_3'].iloc[0]) +
                         len(df_train['embedding_document_id_4'].iloc[0]) +
                         len(df_train['embedding_document_id_5'].iloc[0]) +
                         len(df_train['embedding_full_text'].iloc[0]) +
                         len(df_train['bert_embedding'].iloc[0]) 
                       ))

In [198]:
start_category = 12
end_category = start_category + len(df_train['embedding_category'].iloc[0])
start_tag = end_category
end_tag = start_tag + len(df_train['embedding_tags'].iloc[0])
start_authors = end_tag
end_authors = start_authors + len(df_train['embedding_authors'].iloc[0])
start_titles = end_authors
end_titles = start_titles + len(df_train['embedding_title'].iloc[0])
start_document_id_1 = end_titles
end_document_id_1 = start_document_id_1 + len(df_train['embedding_document_id_1'].iloc[0])
start_document_id_2 = end_document_id_1
end_document_id_2 = start_document_id_2 + len(df_train['embedding_document_id_2'].iloc[0])
start_document_id_3 = end_document_id_2
end_document_id_3 = start_document_id_3 + len(df_train['embedding_document_id_3'].iloc[0])
start_document_id_4 = end_document_id_3
end_document_id_4 = start_document_id_4 + len(df_train['embedding_document_id_4'].iloc[0])
start_document_id_5 = end_document_id_4
end_document_id_5 = start_document_id_5 + len(df_train['embedding_document_id_5'].iloc[0])
start_full_text = end_document_id_5 
end_full_text = start_full_text + len(df_train['embedding_full_text'].iloc[0])
start_bert = end_full_text
end_bert = start_bert + len(df_train['bert_embedding'].iloc[0])

In [199]:
features_all[:, 0] = df_train['ctr'].tolist()
features_all[:, 1] = df_train['day_month'].tolist()
features_all[:, 2] = df_train['month'].tolist()
features_all[:, 3] = df_train['year'].tolist()
features_all[:, 4] = df_train['hour'].tolist()
features_all[:, 5] = df_train['number_week'].tolist()
features_all[:, 6] = df_train['day_week'].tolist()
features_all[:, 7] = df_train['number_title'].tolist()
features_all[:, 8] = df_train['number_text'].tolist()
features_all[:, 9] = df_train['number_authors'].tolist()
features_all[:, 10] = df_train['number_tags'].tolist()
features_all[:, 11] = df_train['curs'].tolist()
features_all[:, start_category:end_category] = df_train['embedding_category'].tolist()
features_all[:, start_tag:end_tag] = df_train['embedding_tags'].tolist()
features_all[:, start_authors:end_authors] = df_train['embedding_authors'].tolist()
features_all[:, start_titles:end_titles] = df_train['embedding_title'].tolist()
features_all[:, start_document_id_1:end_document_id_1] = df_train['embedding_document_id_1'].tolist()
features_all[:, start_document_id_2:end_document_id_2] = df_train['embedding_document_id_2'].tolist()
features_all[:, start_document_id_3:end_document_id_3] = df_train['embedding_document_id_3'].tolist()
features_all[:, start_document_id_4:end_document_id_4] = df_train['embedding_document_id_4'].tolist()
features_all[:, start_document_id_5:end_document_id_5] = df_train['embedding_document_id_5'].tolist()
features_all[:, start_full_text:end_full_text] = df_train['embedding_full_text'].tolist()
features_all[:, start_bert:end_bert] = df_train['bert_embedding'].tolist()

In [200]:
features_all.shape

(6995, 2446)

In [201]:
y = df_train[["views","depth","full_reads_percent"]]

In [202]:
parametrs_catboost = {}
parametrs_catboost['views_0'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0.0,
  'od_type': 'Iter',  
  'learning_rate': 0.05,
  'colsample_bylevel': 0.7, 
  'subsample': 0.7,
  'max_depth': 5,
  'eval_metric':'R2', 
  'verbose':False,
  'early_stopping_rounds': 5
}
parametrs_catboost['views_1'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0.5,
  'od_type': 'Iter',  
  'learning_rate': 0.05,
  'colsample_bylevel': 0.7, 
  'subsample': 0.6,
  'max_depth': 6,
  'eval_metric':'R2', 
  'verbose':False,
  'early_stopping_rounds': 5
}
parametrs_catboost['views_2'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0.1,
  'od_type': 'Iter',  
  'learning_rate': 0.05,
  'colsample_bylevel': 0.7, 
  'subsample': 0.75,
  'max_depth': 6,
  'eval_metric':'R2', 
  'verbose':False,
  'early_stopping_rounds': 5
}
parametrs_catboost['views_3'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0.05,
  'od_type': 'Iter',  
  'learning_rate': 0.035,
  'colsample_bylevel': 0.6, 
  'subsample': 0.7,
  'max_depth': 6,
  'eval_metric':'R2', 
  'verbose':False,
  'early_stopping_rounds': 10
}

parametrs_catboost['depth_0'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0.0,
  'od_type': 'Iter',  
  'learning_rate': 0.01,
  'colsample_bylevel': 0.65, 
  'subsample': 0.65,
  'max_depth': 6,
  'eval_metric':'R2', 
  'verbose':False,
  'early_stopping_rounds': 10
}
parametrs_catboost['depth_1'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0.1,
  'od_type': 'Iter',  
  'learning_rate': 0.04,
  'colsample_bylevel': 0.65, 
  'subsample': 0.7,
  'max_depth': 5,
  'eval_metric':'R2', 
  'verbose':False,
  'early_stopping_rounds': 5
}
parametrs_catboost['depth_2'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0.1,
  'od_type': 'Iter',  
  'learning_rate': 0.05,
  'colsample_bylevel': 0.7, 
  'subsample': 0.6,
  'max_depth': 4,
  'eval_metric':'R2', 
  'verbose':False,
  'early_stopping_rounds': 5
}
parametrs_catboost['depth_3'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0.0,
  'od_type': 'Iter',  
  'learning_rate': 0.03,
  'colsample_bylevel': 0.6, 
  'subsample': 0.7,
  'max_depth': 6,
  'eval_metric':'R2', 
  'verbose':False,
  'early_stopping_rounds': 10
}
parametrs_catboost['full_reads_percent_0'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0,
  'od_type': 'Iter',  
  'learning_rate': 0.01,
  'colsample_bylevel': 0.5, 
  'subsample': 0.6,
  'max_depth': 3,
  'eval_metric':'R2', 
  'verbose': False,
  'early_stopping_rounds': 5
}

parametrs_catboost['full_reads_percent_1'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0,
  'od_type': 'Iter',  
  'learning_rate': 0.03,
  'colsample_bylevel': 0.5, 
  'subsample': 0.5,
  'max_depth': 4,
  'eval_metric':'R2', 
  'verbose': False,
  'early_stopping_rounds': 7
}

parametrs_catboost['full_reads_percent_2'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0,
  'od_type': 'Iter',  
  'learning_rate': 0.01,
  'colsample_bylevel': 0.5, 
  'subsample': 0.5,
  'max_depth': 3,
  'eval_metric':'R2', 
  'verbose': False,
  'early_stopping_rounds': 7
}

parametrs_catboost['full_reads_percent_3'] = {
  'iterations': 3000, 
  'l2_leaf_reg': 0,
  'od_type': 'Iter',  
  'learning_rate': 0.01,
  'colsample_bylevel': 0.5, 
  'subsample': 0.5,
  'max_depth': 4,
  'eval_metric':'R2', 
  'verbose': False,
  'early_stopping_rounds':7
}



In [203]:
%%time
for fold in range(4):
    with open(f'utils/splits/test_fold_{fold}.pkl', 'rb') as handle:
        test_index = pickle.load(handle)
    with open(f'utils/splits/train_fold_{fold}.pkl', 'rb') as handle:
        train_index = pickle.load(handle)
    pred_all_reg = np.zeros((len(test_index), 3))
    for number_kind, target in enumerate(['views','depth','full_reads_percent']):
        if target == 'views' and fold in [0, 3]:
            regr = catboost.CatBoostRegressor(**parametrs_catboost[f"{target}_{fold}"])
            X_train = features_all[train_index, :start_bert]
            y_train = y.iloc[train_index]
            X_test = features_all[test_index, :start_bert]
            y_test = y.iloc[test_index] 
        
        elif target == 'views' and fold in [1, 2]:
            regr = catboost.CatBoostRegressor(**parametrs_catboost[f"{target}_{fold}"])
            X_train = features_all[train_index, :start_full_text]
            y_train = y.iloc[train_index]
            X_test = features_all[test_index, :start_full_text]
            y_test = y.iloc[test_index] 

        elif target == 'depth' and fold in [0, 3]:
            regr = catboost.CatBoostRegressor(**parametrs_catboost[f"{target}_{fold}"])
            X_train = features_all[train_index, :start_bert]
            y_train = y.iloc[train_index]
            X_test = features_all[test_index, :start_bert]
            y_test = y.iloc[test_index]
            
        elif target == 'depth' and fold in [1, 2]:
            regr = catboost.CatBoostRegressor(**parametrs_catboost[f"{target}_{fold}"])
            X_train = features_all[train_index]
            y_train = y.iloc[train_index]
            X_test = features_all[test_index]
            y_test = y.iloc[test_index]
            
        elif target == 'full_reads_percent':
            regr = catboost.CatBoostRegressor(**parametrs_catboost[f"{target}_{fold}"])
            X_train = features_all[train_index]
            y_train = y.iloc[train_index]
            X_test = features_all[test_index]
            y_test = y.iloc[test_index]
            
        train_pool = catboost.Pool(X_train, y_train[target])
        test_pool = catboost.Pool(X_test, y_test[target])
        regr.fit(train_pool, eval_set=test_pool, use_best_model=True, plot=True)
        pred = regr.predict(X_test)
        pred_all_reg[:, number_kind] = pred
        if target == 'views': 
            regr.save_model(f'models_catboost/_{fold}_views_rbk_model')
        elif target == 'depth': 
            regr.save_model(f'models_catboost/_{fold}_depth_rbk_model')
        elif target == 'full_reads_percent': 
            regr.save_model(f'models_catboost/_{fold}_full_reads_percent_rbk_model')
        print(target)
        print('influence ctr', regr.feature_importances_[0])
        print('influence day_month', regr.feature_importances_[1])
        print('influence month', regr.feature_importances_[2])
        print('influence year', regr.feature_importances_[3])
        print('influence hour', regr.feature_importances_[4])
        print('influence number_week', regr.feature_importances_[5])
        print('influence day_week', regr.feature_importances_[6])
        print('influence number_title', regr.feature_importances_[7])
        print('influence number_text', regr.feature_importances_[8])
        print('influence number_authors', regr.feature_importances_[9])
        print('influence number_tags', regr.feature_importances_[10])
        print('influence curs', regr.feature_importances_[11])
        print('influence embedding_category', regr.feature_importances_[start_category:end_category].sum())
        print('influence embedding_tags', regr.feature_importances_[start_tag:end_tag].sum())
        print('influence embedding_authors', regr.feature_importances_[start_authors:end_authors].sum())
        print('influence embedding_title', regr.feature_importances_[start_titles:end_titles].sum())
        print('influence embedding_document_id_1', regr.feature_importances_[start_document_id_1:end_document_id_1].sum())
        print('influence embedding_document_id_2', regr.feature_importances_[start_document_id_2:end_document_id_2].sum())
        print('influence embedding_document_id_3', regr.feature_importances_[start_document_id_3:end_document_id_3].sum())
        print('influence embedding_document_id_4', regr.feature_importances_[start_document_id_4:end_document_id_4].sum())
        print('influence embedding_document_id_5', regr.feature_importances_[start_document_id_5:end_document_id_5].sum())
        print('influence embedding_full_text', regr.feature_importances_[start_full_text:end_full_text].sum())
        print('influence bert_embedding', regr.feature_importances_[start_bert:end_bert].sum())
    mertic_for_task(y_test, pred_all_reg)
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

views
influence ctr 13.536591192834896
influence day_month 0.35748994422510005
influence month 2.840307538164184
influence year 0.0
influence hour 0.3591759939062804
influence number_week 2.075170719178977
influence day_week 0.0
influence number_title 0.002611502336368397
influence number_text 0.8874151367200153
influence number_authors 0.44517894778048234
influence number_tags 0.3216934127189012
influence curs 7.439201912192198
influence embedding_category 0.00932063438241406
influence embedding_tags 2.47562062724007
influence embedding_authors 1.4104376201242033
influence embedding_title 19.285117322989947
influence embedding_document_id_1 0.04313384177363183
influence embedding_document_id_2 0.07049102062743481
influence embedding_document_id_3 0.8446176570468985
influence embedding_document_id_4 12.550091338802432
influence embedding_document_id_5 0.0
influence embedding_full_text 35.04633363695561
influence bert_embedding 0.0


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

depth
influence ctr 1.2351464747978607
influence day_month 1.216233294494677
influence month 12.11405290150437
influence year 0.006666963412109641
influence hour 0.5215872640211158
influence number_week 38.64118506873655
influence day_week 0.13903492416694613
influence number_title 0.054107912973581196
influence number_text 1.443421970606702
influence number_authors 1.5106751598791577
influence number_tags 0.2978832139225554
influence curs 4.612191009381126
influence embedding_category 0.16925082623184465
influence embedding_tags 1.6779553218006815
influence embedding_authors 3.3262782126801294
influence embedding_title 7.30795485560558
influence embedding_document_id_1 2.0174296696844065
influence embedding_document_id_2 0.7027274795657424
influence embedding_document_id_3 5.5271391884149335
influence embedding_document_id_4 4.583980172041741
influence embedding_document_id_5 0.08241908969685204
influence embedding_full_text 12.812679026381414
influence bert_embedding 0.0


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

...

views
influence ctr 10.598205637822272
influence day_month 0.2434483084851873
influence month 2.580486988750771
influence year 0.0
influence hour 0.6518049609745027
influence number_week 1.6785144615729
influence day_week 0.06944135428337368
influence number_title 0.016123104705128285
influence number_text 1.0836925112746985
influence number_authors 0.7808303600065152
influence number_tags 0.006413070604980755
influence curs 5.251285485894354
influence embedding_category 0.04712231961112982
influence embedding_tags 11.504939108318158
influence embedding_authors 0.7663215360509044
influence embedding_title 5.48082807701551
influence embedding_document_id_1 0.1289718235656758
influence embedding_document_id_2 0.11375898869745238
influence embedding_document_id_3 0.49605016238789595
influence embedding_document_id_4 18.186491812643485
influence embedding_document_id_5 0.0
influence embedding_full_text 40.31526992733508
influence bert_embedding 0.0


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

depth
influence ctr 1.3640893571350294
influence day_month 1.0117414711559536
influence month 18.376917531039716
influence year 0.0
influence hour 0.4805876009718478
influence number_week 27.974994020493334
influence day_week 0.29643047028231617
influence number_title 0.028101620958293674
influence number_text 1.0467319616440138
influence number_authors 1.983127489202605
influence number_tags 1.254439236995294
influence curs 7.279623293940065
influence embedding_category 0.10143292762444125
influence embedding_tags 3.6678126818284857
influence embedding_authors 3.051390767530206
influence embedding_title 4.797034542003314
influence embedding_document_id_1 1.199286611362286
influence embedding_document_id_2 0.4370179448234501
influence embedding_document_id_3 4.618667722999314
influence embedding_document_id_4 4.046709286685392
influence embedding_document_id_5 0.15614042018775384
influence embedding_full_text 16.827723041136508
influence bert_embedding 0.0


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

full_reads_percent
influence ctr 8.431714303411072
influence day_month 0.003009639596495056
influence month 0.08343602056136905
influence year 0.0
influence hour 1.3513709045543307
influence number_week 0.35157945350514885
influence day_week 0.037341395775853686
influence number_title 0.005187502167921233
influence number_text 2.428797499331698
influence number_authors 7.502105195131393
influence number_tags 0.41408045799164106
influence curs 2.0911529386818444
influence embedding_category 0.2077479425571731
influence embedding_tags 1.8315866216529075
influence embedding_authors 0.15560884799618815
influence embedding_title 2.411396354532365
influence embedding_document_id_1 0.0
influence embedding_document_id_2 0.03015018374364071
influence embedding_document_id_3 0.22577351795768089
influence embedding_document_id_4 0.9496859006827753
influence embedding_document_id_5 0.005267726266640754
influence embedding_full_text 5.331097109837625
influence bert_embedding 66.15191048406439
score

### Inference

In [222]:
df_test = pd.read_parquet('dataset/test_dataset_test_full_proc.parquet')

In [223]:
features_all = np.zeros((3000, 12 + 
                         len(df_test['embedding_category'].iloc[0]) + 
                         len(df_test['embedding_tags'].iloc[0]) + 
                         len(df_test['embedding_authors'].iloc[0]) + 
                         len(df_test['embedding_title'].iloc[0]) + 
                         len(df_test['embedding_document_id_1'].iloc[0]) +
                         len(df_test['embedding_document_id_2'].iloc[0]) +
                         len(df_test['embedding_document_id_3'].iloc[0]) +
                         len(df_test['embedding_document_id_4'].iloc[0]) +
                         len(df_test['embedding_document_id_5'].iloc[0]) +
                         len(df_test['embedding_full_text'].iloc[0]) +
                         len(df_test['bert_embedding'].iloc[0]) 
                       ))

In [224]:
start_category = 12
end_category = start_category + len(df_test['embedding_category'].iloc[0])
start_tag = end_category
end_tag = start_tag + len(df_test['embedding_tags'].iloc[0])
start_authors = end_tag
end_authors = start_authors + len(df_test['embedding_authors'].iloc[0])
start_titles = end_authors
end_titles = start_titles + len(df_test['embedding_title'].iloc[0])
start_document_id_1 = end_titles
end_document_id_1 = start_document_id_1 + len(df_test['embedding_document_id_1'].iloc[0])
start_document_id_2 = end_document_id_1
end_document_id_2 = start_document_id_2 + len(df_test['embedding_document_id_2'].iloc[0])
start_document_id_3 = end_document_id_2
end_document_id_3 = start_document_id_3 + len(df_test['embedding_document_id_3'].iloc[0])
start_document_id_4 = end_document_id_3
end_document_id_4 = start_document_id_4 + len(df_test['embedding_document_id_4'].iloc[0])
start_document_id_5 = end_document_id_4
end_document_id_5 = start_document_id_5 + len(df_test['embedding_document_id_5'].iloc[0])
start_full_text = end_document_id_5 
end_full_text = start_full_text + len(df_test['embedding_full_text'].iloc[0])
start_bert = end_full_text
end_bert = start_bert + len(df_test['bert_embedding'].iloc[0])

In [225]:
features_all[:, 0] = df_test['ctr'].tolist()
features_all[:, 1] = df_test['day_month'].tolist()
features_all[:, 2] = df_test['month'].tolist()
features_all[:, 3] = df_test['year'].tolist()
features_all[:, 4] = df_test['hour'].tolist()
features_all[:, 5] = df_test['number_week'].tolist()
features_all[:, 6] = df_test['day_week'].tolist()
features_all[:, 7] = df_test['number_title'].tolist()
features_all[:, 8] = df_test['number_text'].tolist()
features_all[:, 9] = df_test['number_authors'].tolist()
features_all[:, 10] = df_test['number_tags'].tolist()
features_all[:, 11] = df_test['curs'].tolist()
features_all[:, start_category:end_category] = df_test['embedding_category'].tolist()
features_all[:, start_tag:end_tag] = df_test['embedding_tags'].tolist()
features_all[:, start_authors:end_authors] = df_test['embedding_authors'].tolist()
features_all[:, start_titles:end_titles] = df_test['embedding_title'].tolist()
features_all[:, start_document_id_1:end_document_id_1] = df_test['embedding_document_id_1'].tolist()
features_all[:, start_document_id_2:end_document_id_2] = df_test['embedding_document_id_2'].tolist()
features_all[:, start_document_id_3:end_document_id_3] = df_test['embedding_document_id_3'].tolist()
features_all[:, start_document_id_4:end_document_id_4] = df_test['embedding_document_id_4'].tolist()
features_all[:, start_document_id_5:end_document_id_5] = df_test['embedding_document_id_5'].tolist()
features_all[:, start_full_text:end_full_text] = df_test['embedding_full_text'].tolist()
features_all[:, start_bert:end_bert] = df_test['bert_embedding'].tolist()

In [226]:
for kind in ["views", "depth", "full_reads_percent"]:
    prediction = np.zeros((4, 3000))
    for fold in range(4):
        model_path = f"models_catboost/_{fold}_{kind}_rbk_model"
        model = catboost.CatBoostRegressor()
        model.load_model(model_path)
        if kind=='views':
            if fold in [0,3]:
                prediction[fold, :] = model.predict(features_all[:, :start_bert])
            elif fold in [1,2]:
                prediction[fold, :] = model.predict(features_all[:, :start_full_text])
        elif kind=='depth':
            if fold in [0,3]:
                prediction[fold, :] = model.predict(features_all[:, :start_bert])
            elif fold in [1,2]:
                prediction[fold, :] = model.predict(features_all)
        elif kind=='full_reads_percent':
            prediction[fold, :] = model.predict(features_all)
    df_test[kind] = np.mean(prediction, axis=0) 

In [227]:
df_test['views'] = df_test['views'].apply(lambda x: round(x))
df_test['depth'] = df_test['depth'].apply(lambda x: round(x, 3))
df_test['full_reads_percent'] = df_test['full_reads_percent'].apply(lambda x: round(x, 3))

In [228]:
df_test.reset_index(inplace=True)

In [229]:
df_test.head()

,level_0,document_id,title,publish_date,session,authors,ctr,category,tags,full_text,...,embedding_document_id_4,embedding_document_id_5,embedding_tags,embedding_authors,embedding_title,embedding_full_text,bert_embedding,views,depth,full_reads_percent
0,0,61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,крайне провокационная тема россия сша поспорил...,2022-02-01,J0AvX96vTAaQCiWVbzoMdw,,0.000,5409f11ce063da9c8b588a12,58ca77679a794700b1e37f0a 5409f1b7e063daa0f408a...,the washington post пишет перепалка дипломатам...,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[-0.3261207789182663, 0.029807353392243385, -0...",19541,1.181,45.133
1,1,628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,власти херсонской области пообещали сделать ру...,2022-05-24,QS5CqzXYRnmDdR2LaSreEw,,1.598,5409f11ce063da9c8b588a12,5433603acbb20f6e5def0cc5 5409f420e063daa0f408b...,русский язык наравне украинским получит статус...,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[-0.30430955812335014, -0.02844966948032379, -...",7332,1.059,30.765
2,2,627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,премьер президент финляндии призвали срочно по...,2022-05-12,XVsoyrUOT8OJJg2_finJhw,,2.006,5409f11ce063da9c8b588a12,5409f270e063daa0f408b0a7 5409f196e063daa0f408a...,почему финляндия решила вступать натопрезидент...,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[-0.30152420327067375, 0.004645545035600662, -...",9664,1.057,27.600
3,3,628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,госдуму внесли проект временных администрациях...,2022-05-19,upfii3whSSuMXCqcqF8VbQ,,0.000,5409f11ce063da9c8b588a12,5409f145e063daa0f408ad35 5409f30fe063daa0f408b...,группа депутатов фракции справедливая россия п...,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[-0.27513759955763817, -0.0069693028926849365,...",5892,1.063,32.833
4,4,620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,блинкен заявил оон готовности россии начать вт...,2022-02-17,5hzCiIHdSYKQIr8WAM18bw,604f2c199a79478ec0029eae,9.136,5409f11ce063da9c8b588a12,5409f15de063daa0f408ada8 545d4dcdcbb20fbc56178...,российские войска готовятся начать вторжение у...,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, ...","[-0.2792944088578224, 0.04815381579101086, -0....",99693,1.139,31.154


In [230]:
df_test[['document_id', 'views', 'depth', 'full_reads_percent']].to_csv("submission.csv", index=False)